# [Project Euler Problem 54](https://projecteuler.net/problem=54)

In [148]:
class PokerCard():

    rank_lu = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':11, 'Q':12, 'K':13, 'A':14}
    suits = {'C', 'D', 'H', 'S'}

    def __init__(self, card):
        self.face = card[0]
        self.suit = card[1]
        if not (self.face in PokerCard.rank_lu and self.suit in PokerCard.suits):
            raise ValueError('invalid card')

    def rank(self):
        return PokerCard.rank_lu[self.face]

    def __repr__(self):
        return f'card({self.face}{self.suit})'

    def __str__(self):
        return self.face + self.suit

    def __eq__(self, other):
        return self.face == other.face and self.suit == other.suit

    def __ge__ (self, other):
        return self.rank() >= other.rank()

    def __gt__ (self, other):
        return self.rank() > other.rank()

In [206]:
class PokerHand():

    _types = ['nothing', 'pair', 'two pair', 'three of a kind', 'straight', 'flush', 'full house', 'four of a kind', 'straight flush']
    _rank_lu = dict([(h,r) for r,h in enumerate(_types)])
    _type_lu = dict(enumerate(_types))
                  
    def __init__(self, cards):
        if type(cards) is list:
            if type(cards[0]) is PokerCard:
                self.cards = cards
            elif type(cards[0]) is str:
                self.cards = [PokerCard(c) for c in cards]
            else:
                raise ValueError('Unrecognized card format')
        elif type(cards) is str:
            self.cards = [PokerCard(c) for c in cards.rstrip().split()]
        else:
            raise ValueError('Unrecognized card format')
        if len(self.cards) != 5:
                raise ValueError('A poker hand must contain 5 cards')
        self.cards.sort()
        self.ranks = [c.rank() for c in self.cards]

    def is_flush(self):
        suit = self.cards[0].suit
        for c in self.cards[1:]:
            if c.suit != suit:
                return False
        return True

    def is_straight(self):
        if self.ranks == [2,3,4,5,14]:
            return True
        for k in range(4):
            if self.ranks[k+1] != self.ranks[k] + 1:
                return False
        return True

    def is_straight_flush(self):
        return self.is_flush() and self.is_flush()

    def is_four_of_a_kind(self):
        return self.ranks.count(self.ranks[1]) == 4

    def is_full_house(self):
        c1,c5 = [self.ranks.count(c) for c in self.ranks[::4]]
        return c1 == 2 and c5 == 3 or c1 == 3 and c5 == 2

    def is_three_of_a_kind(self):
        return self.ranks.count(self.ranks[2]) == 3

    def is_two_pair(self):
        r = self.ranks
        return r[0]==r[1] and (r[2]==r[3] or r[3]==r[4]) or r[1]==r[2] and r[3]==r[4]

    def is_pair(self):
        for k in range(4):
            if self.ranks[k+1] == self.ranks[k]:
                 return True
        return False

    def straight_rank(self):
        if not self.is_straight():
            return 0
        elif self.ranks[0] == 2 and self.ranks[-1] == 14:
            return 1
        return self.ranks[0]
    
    def high_card_rank(self):
        return max([r for r in self.ranks if self.ranks.count(r) == 1])

    def high_pair_rank(self):
        pair_ranks = [r for r in self.ranks if self.ranks.count(r) == 2]
        if pair_ranks:
            return max(pair_ranks)
        return 0

    def low_pair_rank(self):
        pair_ranks = [r for r in self.ranks if self.ranks.count(r) == 2]
        if pair_ranks:
            return min(pair_ranks)
        return 0
    
    def hand_type(self):
        if self.is_straight_flush():
            return 'straight flush'
        elif self.is_four_of_a_kind():
            return 'four of a kind'
        elif self.is_full_house():
            return 'full house'
        elif self.is_flush():
            return 'flush'
        elif self.is_straight():
            return 'straight'
        elif self.is_three_of_a_kind():
            return 'three of a kind'
        elif self.is_two_pair():
            return 'two pair'
        elif self.is_pair():
            return 'pair'
        return 'nothing'
    
    def hand_score(self):
        if self.is_straight_flush():
            return (self.__class__._rank_lu['straight flush'], self.straight_rank())
        elif self.is_four_of_a_kind():
            return (self.__class__._rank_lu['four of a kind'], self.ranks[2])
        elif self.is_full_house():
            return (self.__class__._rank_lu['full house'], self.ranks[2])
        elif self.is_flush():
            return tuple([self.__class__._rank_lu['flush']] + self.ranks[::-1])
        elif self.is_straight():
            return (self.__class__._rank_lu['straight'], self.straight_rank())
        elif self.is_three_of_a_kind():
            return (self.__class__._rank_lu['three of a kind'], self.ranks[2])
        elif self.is_two_pair():
            return (self.__class__._rank_lu['two pair'], self.high_pair_rank(), self.low_pair_rank(), self.high_card_rank())
        elif self.is_pair():
            return tuple([self.__class__._rank_lu['pair'], self.high_pair_rank()] + [r for r in self.ranks[::-1] if self.ranks.count(r) == 1])
        return tuple([self.__class__._rank_lu['nothing']] + self.ranks[::-1])

    def __repr__(self):
        return ' '.join([str(c) for c in self.cards]) + f' ({self.hand_type()})'

    def __eq__(self, other):
        return self.ranks == other.ranks

    def __gt__(self, other):
        sc1 = self.hand_score()
        sc2 = other.hand_score()
        if sc1[0] != sc2[0]:
            return sc1[0] > sc2[0]
        else:
            for r1,r2 in zip(sc1[1:],sc2[1:]):
                if r1 != r2:
                    return r1 > r2
        return False

In [207]:
cnt = 0
for ln in open('p054_poker.txt', 'r'):
    cards = ln.strip().split()
    if PokerHand(cards[:5]) > PokerHand(cards[5:]):
        cnt += 1
cnt

376

In [208]:
cards = ['8S', '3C', '4S', 'TS', '7S', '4D', '5C', '2S', '6H', '7C']
PokerHand(cards[:5])

3C 4S 7S 8S TS (nothing)

In [209]:
PokerHand(['AC', 'AD', 'AH', 'AS', 'KH'])

KH AC AD AH AS (four of a kind)

In [210]:
PokerHand('AC AD AH AS KH')

KH AC AD AH AS (four of a kind)